From multiprocessing import Pool

import grpc
from google.protobuf.json_format import Parse

from insight.proto import bayesian_pb2, bayesian_pb2_grpc

query = None
with open('query.json', 'r') as f:
    query = f.read()
    f.close()

message = Parse(query, bayesian_pb2.QueryId(), ignore_unknown_fields=False)

channel = grpc.insecure_channel('localhost:7001')
stub = bayesian_pb2_grpc.BayesNetStub(channel)


# response = stub.AskNet(message)
def ask_net():
    print('calling')
    stub.AskNet(message)


if __name__ == '__main__':
    pool = Pool()
    result1 = pool.apply_async(ask_net)  # evaluate "solve1(A)" asynchronously
    result2 = pool.apply_async(ask_net)  # evaluate "solve2(B)" asynchronously
    answer1 = result1.get(timeout=100)
    answer2 = result2.get(timeout=100)
    print('completed')

In [1]:
import pickle
from os.path import exists
import sn_bayes
from sn_bayes import longevity_bayes

In [2]:


if exists("bayesianNetwork.pkl"):
        infile = open("bayesianNetwork.pkl",'rb')
        bayesianNetwork = pickle.load(infile)
        infile.close()
 


In [3]:

import sn_bayes


import sn_service.service_spec.bayesian_pb2
from sn_service.service_spec.bayesian_pb2 import BayesianNetwork
from sn_bayes.utils import get_var_val_positions
from sn_bayes.utils import get_var_positions
import sn_service.service_spec.bayesian_pb2
from sn_service.service_spec.bayesian_pb2 import Query
var_positions = get_var_positions(bayesianNetwork)
var_val_positions = get_var_val_positions(bayesianNetwork)
evidence = {}

outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations","hallmark_4_loss_of_proteostasis",
        "hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction","hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion",
        "hallmark_9_altered_intercellular_communication","hallmark_10_extracellular_matrix_dysfunction","poor_diet",
        "poor_diet_flag","poor_sleep","psychological_disorders","obesity","lack_of_exercise","poor_diet_quantity","smoking"]
explainvars=outvars

include_list =["lack_of_exercise_signal","resting_heart_rate","signal_heart_disorders_indicators","steps_anomaly","workout_anomaly","heart_rate_anomaly","step_asymmetry_anomaly","step_variability_anomaly",
 "stride_variability_anomaly","heart_rate_variability_anomaly","walking_speed_variability_anomaly"]


query = Query()
query.switch = "internal_query"

for v in include_list:
    if v in var_positions:
        outvar = query.include_list.add()
        outvar.var_num = var_positions[v]
for var,val  in  evidence.items():
    e= query.evidence.add()
    e.var_num = var_positions[var] if var in var_positions else 0
    e.response = var_val_positions[var][val] if var in var_positions and val in var_val_positions[var] else 0
for o in outvars:
    outvar = query.outvars.add()
    outvar.var_num = var_positions[o] if o in var_positions else 0


In [4]:
from sn_bayes.utils import create_query
import grpc
import sn_service.service_spec.bayesian_pb2_grpc as grpc_bayes_grpc
import sn_service.service_spec.bayesian_pb2
from sn_service.service_spec.bayesian_pb2 import BayesianNetworkQuery
from sn_service.service_spec.bayesian_pb2 import QueryId
from sn_service.service_spec.bayesian_pb2 import Id

endpoint = "localhost:7003"
channel = grpc.insecure_channel("{}".format(endpoint))
stub = grpc_bayes_grpc.BayesNetStub(channel)

print ("Calling StartNet")
%time response = stub.StartNet(bayesianNetwork)
print("response.id")
print(response.id)
print("response.error_msg")
print(response.error_msg)

Calling StartNet
CPU times: user 778 ms, sys: 529 ms, total: 1.31 s
Wall time: 2min 6s
response.id
594e7aa9-f17f-4cf8-ad82-2ee56cc96319
response.error_msg



In [5]:
from multiprocessing import Pool


queryId = QueryId()
queryId.id = response.id
queryId.query.CopyFrom(query)

# response = stub.AskNet(message)
def ask_net():
    print('calling')
    stub.AskNet(queryId)


if __name__ == '__main__':
    pool = Pool()
    result1 = pool.apply_async(ask_net)  # evaluate "solve1(A)" asynchronously
    result2 = pool.apply_async(ask_net)  # evaluate "solve2(B)" asynchronously
    answer1 = result1.get(timeout=100)
    answer2 = result2.get(timeout=100)
    print('completed')

calling
calling


MaybeEncodingError: Error sending result: '<multiprocessing.pool.ExceptionWithTraceback object at 0x7f600c67d400>'. Reason: 'TypeError("can't pickle _thread.RLock objects",)'